In [ ]:
import numpy 

In [ ]:
import torch
import math

In [ ]:
from scipy.integrate import quad

In [ ]:
def fint(x):
  return torch.tensor([0.5])*(torch.tensor([1.0])+torch.erf(x/(torch.sqrt(torch.tensor([2.0]))*R)))/(torch.tensor([1.0])+torch.tensor([0.01])*torch.log(torch.tensor([1.0])+torch.exp(torch.tensor([x/R-0.5])))**2)
  #return 0.5*(1+math.erf(x/(math.sqrt(2)*R)))/(1+0.01*math.log(1+math.exp(x/R-0.5))**2)

In [ ]:

def yc(x,gamma,i):
  lista_integral=[]
  for j in i:
    val=quad(fint, j*8*R, (j+1)*8*R)[0]
    lista_integral.append(val)
  tensor_vals=torch.tensor(lista_integral)
  
  return tensor_vals

In [ ]:
dcpdgi(gamma)

tensor([-0., -0., -0., -0., -0.], grad_fn=<MulBackward0>)

In [ ]:
n=5 #numero de turbinas


cT=0.8
sgm01=[0.5,0.2,0.6,0.65,0.22]
sgm02=[0.4,0.55,0.53,0.51,0.5]

kw1=[0.095, 0.093, 0.1, 0.13, 0.05]
kw2=[0.04,0.12,0.11,0.1,0.09]

#definimos velocidad inicial
uinf=8
#definimos el radio de las aspas
R=40
yT=torch.tensor([R*1.11])

def dw(x,kw):
  return torch.tensor([1.0])+kw*torch.log(1+torch.exp(torch.tensor([2*(x/(2*R)-1)])))

def cP(gamma):
  return torch.tensor([4.0])*api(gamma)*(1-api(gamma))**2*torch.cos(gamma)**2
def api(x):
  return torch.tensor([0.5])*(1-torch.sqrt(torch.tensor([1.0-cT])))
def aj(gamma):
  return torch.tensor([0.5])*(1-torch.sqrt(1-cT*torch.cos(gamma)**2))
def sig(x,kw,sgm):
  return sgm*dw(x,kw)
#estas son las dificiles

def u(gamma,kw,sgm,i):
  x=8*R
  return torch.tensor([uinf,uinf,uinf,uinf,uinf])-((aj(gamma)*torch.sqrt(torch.tensor([2.0*torch.pi]))*torch.tensor([uinf,uinf,uinf,uinf,uinf])*torch.cos(gamma))/(8*dw(x,kw)*sgm)*(torch.tensor([1.,1.,1.,1.,1.])+torch.erf(8/torch.sqrt(torch.tensor([2.0])))))*(torch.erf((yT+R-yc(x,gamma,i))/(torch.sqrt(torch.tensor([2.0]))*sig(x,kw,sgm)))-torch.erf((yT-R-yc(x,gamma,i))/(torch.sqrt(torch.tensor([2.0]))*sig(x,kw,sgm))))




#definicion de las demas vcariables
def dpdcpi(gamma, kw, sgm,i):
  return 0.5*torch.pi*R**2*torch.cos(gamma)**2*u(gamma,kw, sgm,i)**3

def dcpdgi(gamma): #dCp/d\gamma
  return torch.tensor([-8.0])*api(gamma)*(torch.tensor([1.0])-api(gamma))**2*torch.sin(gamma)*torch.cos(gamma)

def dcpdapi(gamma): #dCp/d\gamma
  return 4*(torch.tensor([1.0])-4*api(gamma) +3*api(gamma))*torch.cos(gamma)**2

def dadcti(gamma): #da/dCt
  return 1/(4*torch.sqrt(torch.tensor([1.0-cT])))

def dctdgi(gamma): #dCt/d\gamma
  return torch.tensor([-8.0])*api(gamma)*(torch.tensor([1.0])-api(gamma))*torch.sin(gamma)*torch.cos(gamma)

#para la turbina trasera es decir la j
def dpjduei(gamma, kw, sgm,i):
  return 1.5*math.pi*R**2*torch.cos(gamma)**2*cP(gamma)*u(gamma,kw, sgm,i)**2 #definir cp y u
def dueidyi(gamma, kw, sgm,i):
  #x'=4D
  x=8*R
  return -(uinf*R*torch.cos(gamma)*aj(gamma))/(4*dw(x,kw)**2*sgm**2)*(torch.tensor([1.0])+torch.erf(torch.tensor([8.0])/math.sqrt(torch.tensor([2.0]))))*(torch.exp(-(yT-R-yc(x,gamma,i))**2/(2*sig(x,kw,sgm)**2))-torch.exp(-(yT+R-yc(x,gamma,i))**2/(2*sig(x,kw,sgm)**2)))
  #-(uinf*R*math.cos(gamma)*aj(gamma))/(4*dw(x,kw)**2*sgm**2)*(1+math.erf(8/math.sqrt(2)))*
  #aqui estamos trabajando
  #definir fundiones dw(x), yc(x), aj()

def dyidgi(x,gamma,i):
  
  return (torch.cos(gamma)**3-2*torch.sin(gamma)**2*torch.cos(gamma))*0.25*cT*yc(x,gamma,i)[0]
def duedai(gamma,kw,sgm,i):
  x=8*R
  return -(torch.sqrt(torch.tensor([2.0*torch.pi]))*uinf*R*torch.cos(gamma))/(8*dw(x,kw)*sgm)*(torch.tensor([1.0])+torch.erf(8/torch.sqrt(torch.tensor([2.0]))))*(torch.erf((yT+R-yc(x,gamma,i))/(torch.sqrt(torch.tensor([2.0]))*abs(sig(x,kw,sgm))))-torch.erf((yT-R-yc(x,gamma,i))/(torch.sqrt(torch.tensor([2.0]))*abs(sig(x,kw,sgm)))))

def daidgi(gamma):
  return -(cT*torch.sin(gamma)*torch.cos(gamma))/(2*torch.sqrt(1-cT*torch.cos(gamma)**2))
#ahora definicion de las variables d elos angulos de cada turbina

#el learning rate 
alpha=1e-4






In [ ]:
#las funciones a hacerles el back propagation 
def dpidgi(gamma,kw,sgm,i):
  x=8*R
  return dpdcpi(gamma, kw, sgm,i)*dcpdgi(gamma)+dpdcpi(gamma, kw, sgm,i)*dcpdapi(gamma)*dadcti(gamma)*dctdgi(gamma)
def dpjdgi(gamma,kw,sgm,i):
  x=8*R
  return dpjduei(gamma, kw, sgm,i)*dueidyi(gamma, kw, sgm,i)*dyidgi(x,gamma,i)+dpjduei(gamma, kw, sgm,i)*duedai(gamma,kw,sgm,i)*daidgi(gamma)

In [ ]:
gamma=torch.tensor([0.0,0.0,0.0,0.0,0.0], requires_grad=True)
kw=torch.tensor([0.095, 0.093, 0.1, 0.13, 0.05], requires_grad=False)
sgm=torch.tensor([0.5,0.2,0.6,0.65,0.22], requires_grad=False)
i=torch.tensor([1.0,2.0,3.0,4.0,5.0], requires_grad=False)


In [ ]:
def dP(gamma,kw,sgm,i):
  return 0.5*dpidgi(gamma,kw,sgm,i)+0.5*dpjdgi(gamma,kw,sgm,i)

In [ ]:
y = dpjdgi(gamma,kw,sgm,i)
yy=y.sum()
yy.backward()


In [ ]:
gamma.grad


tensor([ 6254078., 58317780., 33262124., 20478266.,  8438127.])

tensor([-723.1716,  601.0576, 6207.5137, 6127.6279, 2348.5935],
       requires_grad=True)

In [ ]:
P_list=[0.0,1.0]
v=0.0
m=0.0
t=0
while torch.abs(torch.norm(P_list[i+1]-P_list[i]))>1e-8
    y = dP(gamma,kw,sgm,i) 
    yy=y.sum()
     yy.backward()
    gamma.data -= alpha*gamma.grad #here we update the values for the angles
    gamma.grad.zero_()
    t=t+1

IndexError: ignored

tensor([0., 2., 2., 2., 2.])

tensor([-70.8652, -34.1290, -18.9847, -11.8885,  -8.0917])

tensor([0.2764, 0.2764, 0.2764, 0.2764, 0.2764], grad_fn=<MulBackward0>)

In [ ]:
variable = torch.tensor([1.], requires_grad=True) #Defines una variable con respecto a la cual puedas derivar o sacar gradiente
funcion = variable**2-3 #Defines una funcion que dependa de esa variable

In [ ]:
funcion.backward() #Con este método calculamos la derivada o gradiente de la funcion con respecto a todas las variables que tengan requires_grad=True

In [ ]:
variable.grad #La derivada queda guardada en el atributo grad de la variable

tensor([2.])

tensor([-602643.,       0.,       0.,       0.,       0.])

En este ejemplo la función era $x²-3$, por lo cual la derivada es $2x$ lo que es consistente con el resultado. Otro punto a detacar es que cuando requieras sacar derivadas en un bucle o iterativamente, es necesario poner a cero el gradiente en cada itercaión, ya que el gradiente se va sumando y acumulamdo en el atributo grad en cada iteración.

Así mismo, para modificar el valor de una variable con requires_grad= True, es necesario hacerlo de la siguiente forma:

In [ ]:
variable.data[0] = 2. 

Ejemplo: Minimizando la función $x²-3$ con el descenso del gradiente, el resultado debe de ser 0 para $x$

In [ ]:
for i in range(100):
  funcion = variable**2-3
  funcion.backward()
  variable.data -= 0.1*variable.grad
  print(variable)
  print(variable.grad)
  variable.grad.zero_()


tensor([1.6000], requires_grad=True)
tensor([4.])
tensor([1.2800], requires_grad=True)
tensor([3.2000])
tensor([1.0240], requires_grad=True)
tensor([2.5600])
tensor([0.8192], requires_grad=True)
tensor([2.0480])
tensor([0.6554], requires_grad=True)
tensor([1.6384])
tensor([0.5243], requires_grad=True)
tensor([1.3107])
tensor([0.4194], requires_grad=True)
tensor([1.0486])
tensor([0.3355], requires_grad=True)
tensor([0.8389])
tensor([0.2684], requires_grad=True)
tensor([0.6711])
tensor([0.2147], requires_grad=True)
tensor([0.5369])
tensor([0.1718], requires_grad=True)
tensor([0.4295])
tensor([0.1374], requires_grad=True)
tensor([0.3436])
tensor([0.1100], requires_grad=True)
tensor([0.2749])
tensor([0.0880], requires_grad=True)
tensor([0.2199])
tensor([0.0704], requires_grad=True)
tensor([0.1759])
tensor([0.0563], requires_grad=True)
tensor([0.1407])
tensor([0.0450], requires_grad=True)
tensor([0.1126])
tensor([0.0360], requires_grad=True)
tensor([0.0901])
tensor([0.0288], requires_grad=T